In [ ]:
%%capture --no-stderr
%pip install -U langgraph langchain langchain_openai langchain_experimental langsmith pandas

In [ ]:
import getpass
import os


def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

# Optional, add tracing in LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

Please provide your OPENAI_API_KEY··········
Please provide your LANGCHAIN_API_KEY··········
Please provide your TAVILY_API_KEY··········


In [ ]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.tools import PythonREPLTool

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
python_repl_tool = PythonREPLTool()

In [ ]:
from langchain_core.messages import HumanMessage

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["messages"][-1].content, name=name)]}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from typing import Literal

# Define the members and system prompt
members = ["Researcher", "Coder"]
system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

# Define the options for the next step
options = ["FINISH"] + members

# Correct usage of Literal without unpacking
class RouteResponse(BaseModel):
    next: Literal['FINISH', 'Researcher', 'Coder']  # Pass options directly

# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

# Initialize the ChatOpenAI model
llm = ChatOpenAI(model="gpt-4o")

# Define the supervisor agent function
def supervisor_agent(state):
    supervisor_chain = (
        prompt
        | llm.with_structured_output(RouteResponse)
    )
    return supervisor_chain.invoke(state)

# Example usage of supervisor_agent function
# Example state can be defined here based on your use case
# state = {"messages": ... }
# response = supervisor_agent(state)
# print(response)


In [ ]:
import functools
import operator
from typing import Sequence, TypedDict, Annotated

from langchain_core.messages import BaseMessage
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import create_react_agent

# Define the base class for the agent's state
class AgentState(TypedDict):
    # This annotation indicates that new messages will be added to the sequence
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # This field represents where to route next
    next: str

# Create the research agent node
research_agent = create_react_agent(llm, tools=[tavily_tool])
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# Create the coder agent node
# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION. PROCEED WITH CAUTION
code_agent = create_react_agent(llm, tools=[python_repl_tool])
code_node = functools.partial(agent_node, agent=code_agent, name="Coder")

# Define supervisor_chain properly using prompt and llm as per your setup
supervisor_chain = (
    prompt
    | llm.with_structured_output(RouteResponse)  # Ensure RouteResponse is defined
)

# Create the workflow graph and add nodes
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", research_node)
workflow.add_node("Coder", code_node)
workflow.add_node("supervisor", supervisor_chain)  # Now supervisor_chain is defined


In [ ]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.add_edge(START, "supervisor")

graph = workflow.compile()

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI  # Make sure to import ChatOpenAI correctly

# Update your ChatOpenAI instance to use a valid model name
llm = ChatOpenAI(model="gpt-4o")  # Use 'gpt-4' if you have access

# Example usage of the graph
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Code hello world and print it to the terminal")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Coder'}}
----


{'Coder': {'messages': [HumanMessage(content='The code has been executed and printed "Hello, World!" to the terminal.', name='Coder')]}}
----
{'supervisor': {'next': 'FINISH'}}
----


In [ ]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Research on super sampling algorithms and give me code trapping rain water")]},
    {"recursion_limit": 100},
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Researcher'}}
----
{'Researcher': {'messages': [HumanMessage(content='### Research on Super Sampling Algorithms\n\n1. **[Intelligent Algorithm for Super-Sampling](https://link.springer.com/chapter/10.1007/978-3-030-05234-8_10)**:\n   This paper presents a novel framework for super-sampling an image. It introduces intelligent interpolation in the intensity domain for learning-based super-resolution methods, contrasting with traditional antialiasing methods.\n\n2. **[NVIDIA DLSS (Deep Learning Super Sampling)](https://developer.nvidia.com/rtx/dlss)**:\n   NVIDIA DLSS is a neural graphics technology that enhances performance using AI. It creates new frames, displays higher resolutions through image reconstruction, and improves image quality, particularly in ray-traced content.\n\n3. **[Deep Learning Super Sampling Research Paper](https://arxiv.org/pdf/2012.09810)**:\n   This paper discusses a method for video super-resolution (SR) using low-resolution aliased fram